In [1]:
# General imports
import h5py
import numpy as np
import pandas as pd
from typing import Any, Dict
from rich import print as rprint
from rich.pretty import pprint as rpprint
from pickle import load

# Tensorflow + keras imports
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.python import keras
from keras import backend as K
from keras.constraints import max_norm
from keras.layers import (
    Activation,
    Add,
    AveragePooling2D,
    AvgPool2D,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    DepthwiseConv2D,
    Dropout,
    Flatten,
    Input,
    Lambda,
    LayerNormalization,
    MaxPooling2D,
    Permute,
)
from keras.models import Model, Sequential
from scikeras.wrappers import KerasClassifier

# Sklearn imports
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Moabb imports
from moabb.pipelines.utils_deep_model import EEGNet, EEGNet_TC
from moabb.pipelines.features import Resampler_Epoch, Convert_Epoch_Array, StandardScaler_Epoch
from moabb import set_log_level
from moabb.utils import setup_seed
from moabb.evaluations import CrossSessionEvaluation, WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.analysis import analyze

# Unused imports
# from braindecode import EEGClassifier
# from braindecode.models import EEGInception
# from scikeras.wrappers import KerasClassifier
# from sklearn.pipeline import Pipeline, make_pipeline
# from skorch.callbacks import EarlyStopping, EpochScoring
# from skorch.dataset import ValidSplit

# Datasets
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from moabb.datasets import Cho2017, Zhou2016

%load_ext autoreload
%autoreload 3

2024-02-17 18:44:10.731659: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 18:44:10.769629: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 18:44:10.770285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 18:44:11.358677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_r

In [2]:
# Set up reproducibility of Tensorflow
setup_seed(42)

In [3]:
# =================================================================================
# DeepConvNet
# =================================================================================
class KerasDeepConvNet(KerasClassifier):
    """Keras implementation of the Deep Convolutional Network as described in
    [1]_.

    This implementation is taken from code by the Army Research Laboratory (ARL)
    at https://github.com/vlawhern/arl-eegmodels

    We use the original parameter implemented on the paper.

    Note that this implementation has not been verified by the original
    authors.

    References
    ----------
    .. [1] Schirrmeister, R. T., Springenberg, J. T., Fiederer, L. D. J., Glasstetter, M., Eggensperger,
           K., Tangermann, M., ... & Ball, T. (2017). Deep learning with convolutional neural networks
           for EEG decoding and visualization. Human brain mapping, 38(11), 5391-5420.
           https://doi.org/10.1002/hbm.23730

    Notes
    -----
    .. versionadded:: 0.5.0
    """

    def __init__(
        self,
        loss,
        optimizer="Adam",
        epochs=1000,
        batch_size=64,
        verbose=0,
        random_state=None,
        validation_split=0.2,
        history_plot=False,
        path=None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.loss = loss
        if optimizer == "Adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.0009)

        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.validation_split = validation_split
        self.history_plot = history_plot
        self.path = path

    def _keras_build_fn(self, compile_kwargs: Dict[str, Any]):
        input_main = Input(shape=(self.X_shape_[1], self.X_shape_[2], 1))

        block1 = Conv2D(
            25,
            (1, 10),
            input_shape=(self.X_shape_[1], self.X_shape_[2], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(input_main)
        block1 = Conv2D(
            25,
            (self.X_shape_[1], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(block1)
        block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
        block1 = Activation("elu")(block1)
        block1 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block1)
        block1 = Dropout(0.5)(block1)

        block2 = Conv2D(50, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block1
        )
        block2 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
        block2 = Activation("elu")(block2)
        block2 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block2)
        block2 = Dropout(0.5)(block2)

        block3 = Conv2D(100, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block2
        )
        block3 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
        block3 = Activation("elu")(block3)
        block3 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block3)
        block3 = Dropout(0.5)(block3)

        block4 = Conv2D(200, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block3
        )
        block4 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
        block4 = Activation("elu")(block4)
        block4 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block4)
        block4 = Dropout(0.5)(block4)

        flatten = Flatten()(block4)

        dense = Dense(self.n_classes_, kernel_constraint=max_norm(0.5))(flatten)
        softmax = Activation("softmax")(dense)

        model = Model(inputs=input_main, outputs=softmax)
        
        model.compile(loss=compile_kwargs["loss"], optimizer=compile_kwargs["optimizer"])
        
        return model
        

keras_deep_conv_net = KerasDeepConvNet(
    loss="sparse_categorical_crossentropy",
    optimizer="Adam",
    epochs=5,
    batch_size=64,
    verbose=1,
    random_state=None,
    validation_split=0.2,
    history_plot=False,
    path=None,
)
# keras_deep_conv_net.save("keras_deep_conv_net.h5")

2024-02-17 18:44:17.906985: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-17 18:44:17.907690: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
# model_keras_pruned = keras.models.load_model("keras_deep_conv_net.h5")
model_keras = keras_deep_conv_net
# Now we need to instantiate a new SciKeras object since we only saved the Keras model
resampler_epoch = Resampler_Epoch(sfreq=250)
convert_epoch_array = Convert_Epoch_Array()
standardscaler_epoch = StandardScaler_Epoch()
keras_deepconvnet_pipeline = make_pipeline(
    resampler_epoch,
    convert_epoch_array,
    standardscaler_epoch,
    model_keras
)

keras_deepconvnet_pipeline

Pipeline(steps=[('resampler_epoch', Resampler_Epoch(sfreq=250)),
                ('convert_epoch_array', Convert_Epoch_Array()),
                ('standardscaler_epoch', StandardScaler_Epoch()),
                ('kerasdeepconvnet',
                 KerasDeepConvNet(build_fn=None, callbacks=None, class_weight=None, epochs=5, loss='sparse_categorical_crossentropy', metrics=None, model=None, optimizer=<keras.optimizers.adam.Adam object at 0x7fab90bcc390>, run_eagerly=False, shuffle=True, validation_batch_size=None, verbose=1, warm_start=False))])

In [18]:
# Restrict this example only on the first two subject of BNCI2014_001
fat_dataset = FatigueMI()
norm_cho_dataset = NormCho2017()
cho_dataset = Cho2017()

fat_dataset.subject_list = fat_dataset.subject_list[:2]
# norm_cho_dataset.subject_list = norm_cho_dataset.subject_list[:32]
# cho_dataset.subject_list = cho_dataset.subject_list[:32]

datasets = [
    fat_dataset,
    # norm_cho_dataset,
    # cho_dataset
]

evaluation_dict = {
    "WithinSession": WithinSessionEvaluation
}
paradim_dict = {
    "LeftRightImagery": LeftRightImagery
}
pipeline_dict = {
    "Keras_DeepConvNet": keras_deepconvnet_pipeline,
}

results_dict = {}
for model_text in pipeline_dict.keys():
    for evaluation_text in ["WithinSession"]: #, "CrossSession", "CrossSubject"]:
        for paradim_text in ["LeftRightImagery"]: #, "FilterBankMotorImagery"]:
            rprint(f"Evaluating {model_text} using {evaluation_text} and {paradim_text}...")
            evaluation = evaluation_dict[evaluation_text](
                paradigm=paradim_dict[paradim_text](), datasets=datasets, overwrite=True, return_epochs=True,  hdf5_path=f"./saved_models/", save_model=True
            )
            results = evaluation.process({ model_text: pipeline_dict[model_text] })
            results_dict[model_text + "_" + evaluation_text + "_" + paradim_text] = results



Evaluating Keras_DeepConvNet using WithinSession and LeftRightImagery...

FatigueMI-WithinSession:   0%|          | 0/2 [00:00<?, ?it/s]

Adding metadata with 3 columns
Adding metadata with 3 columns
108 matching events found
No baseline correction applied


/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/paradigms/base.py:354: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)


Epoch 1/5


...

/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/pipelines/features.py:163: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return X.get_data()
2024-02-17 20:16:28.244843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [69,20,501]
	 [[{{node Placeholder/_10}}]]
2024-02-17 20:16:28.245045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [69,1]
	 [[{{node Placeholder/_11}}]]


2/2 [==============================] - 2s 201ms/step - loss: 1.2258 - val_loss: 0.6837
Epoch 2/5


2024-02-17 20:16:29.900130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]
2024-02-17 20:16:29.900535: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]


2/2 [==============================] - 0s 41ms/step - loss: 0.7513 - val_loss: 0.6996
Epoch 3/5
2/2 [==============================] - 0s 39ms/step - loss: 0.8558 - val_loss: 0.7198
Epoch 4/5
2/2 [==============================] - 0s 40ms/step - loss: 0.8270 - val_loss: 0.7490
Epoch 5/5
1/1 [==============================] - 0s 99ms/step
Saving Keras model


/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/pipelines/features.py:163: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return X.get_data()
2024-02-17 20:16:30.790349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,501]
	 [[{{node Placeholder/_10}}]]
2024-02-17 20:16:30.790603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,501]
	 [[{{node Placeholder/_10}}]]
FatigueMI-WithinSession: 100

In [20]:
rpprint(
    results_dict['Keras_DeepConvNet_WithinSession_LeftRightImagery']
)

# analyze(
#     results = results_dict['Keras_DeepConvNet_Trained_WithinSession_LeftRightImagery'],
#     out_path = "./saved_models",
#     name = "Keras_DeepConvNet_Trained_WithinSession_LeftRightImagery",
#     plot = True
# )

│     score      time  samples subject session  channels  n_sessions  \
0  0.450909  3.259214    108.0       1       0        20           1   
1  0.542314  3.063900    108.0       2       0        20           1   
│    
│    dataset           pipeline  
0  FatigueMI  Keras_DeepConvNet  
1  FatigueMI  Keras_DeepConvNet

In [21]:

with h5py.File('./saved_models/results/LeftRightImagery/WithinSessionEvaluation/results_.hdf5', 'r') as hfile:
    hfile_group = hfile[list(hfile.keys())[0]]
    id = hfile_group['FatigueMI'].get('id')[()]
    data = hfile_group['FatigueMI'].get('data')[()]
    rpprint({
        "id": id,
        "data": data
    })


{
│   'id': array([[b'1', b'0'],
│      [b'2', b'0']], dtype=object),
│   'data': array([[  0.45090908,   3.2592142 , 108.        ],
│      [  0.54231405,   3.0639    , 108.        ]], dtype=float32)
}

### Pruning & Quantization

In [9]:
trained_model = tf.keras.models.load_model('./saved_models/Models_WithinSession/FatigueMI/1/0/Keras_DeepConvNet/kerasdeepconvnet_fitted_model_0.h5')

In [15]:
model_for_pruning = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.0, final_sparsity=0.5, begin_step=2000, end_step=4000
)

pruned_model = tfmot.sparsity.keras.prune_low_magnitude(trained_model, pruning_schedule=model_for_pruning)

pruned_model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

pruned_model.save('./saved_models/Models_WithinSession/FatigueMI/1/0/Keras_DeepConvNet/pruned_kerasdeepconvnet_fitted_model_0.h5')

pruned_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 501, 1)]      0         
                                                                 
 prune_low_magnitude_conv2d   (None, 20, 492, 25)      527       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_conv2d_  (None, 1, 492, 25)       25027     
 1 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_batch_n  (None, 1, 492, 25)       101       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_activat  (None, 1, 492, 25)       1     

In [ ]:
## Quantization
# converter = tf.lite.TFLiteConverter.from_saved_model('./saved_models/Models_WithinSession/FatigueMI/1/0/Keras_DeepConvNet/pruned_kerasdeepconvnet_fitted_model_0.h5')
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quant_model = converter.convert()

In [16]:
model_keras_pruned = keras.models.load_model(
    "./saved_models/Models_WithinSession/FatigueMI/1/0/Keras_DeepConvNet/pruned_kerasdeepconvnet_fitted_model_0.h5",
)
# # Now we need to instantiate a new SciKeras object since we only saved the Keras model
# resampler_epoch = Resampler_Epoch(sfreq=250)
# convert_epoch_array = Convert_Epoch_Array()
# standardscaler_epoch = StandardScaler_Epoch()
# keras_deepconvnet_pipeline = make_pipeline(
#     resampler_epoch,
#     convert_epoch_array,
#     standardscaler_epoch,
#     model_keras_pruned
# )

# keras_deepconvnet_pipeline

ValueError: Unknown layer: PruneLowMagnitude. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.